<a href="https://colab.research.google.com/github/ohhongseok/ML-DeepLearning/blob/main/7/7-2/%EC%8B%AC%EC%B8%B5_%EC%8B%A0%EA%B2%BD%EB%A7%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>